In [1]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import requests as req

import sys
sys.path.append("/Users/inunezfe/Documents/netCD4")
import state_locations

In [4]:
# Will download the data using the urls from the isimip webpage. 

def get__isimip_data(path):

    url = 'https://files.isimip.org/' + path

    response = req.get(url)

    if response.status_code == 200:
        print('Success!')
    elif response.status_code == 404:
        print('Not Found.')


        return response





In [5]:
'''#### Loads MERRA netcdf file and important variables ####
# Note: operates over one day of data based on the daily nature of each MERRA file # 
def organize_MERRAvar(MERRA_file_path):
    data = Dataset(MERRA_file_path)
    
    ### Extracts location variables and other important variables from "data" ###
    lons = data.variables['lon'][:]
    lats = data.variables['lat'][:]
    hurs = data.variables['hurs'][:,:,:]
    
    # Arturo se la come
    
    ### print the variables saved on the dataset
    print(data.variables)
    
    ### Defines time variable ###
    
    time = np.arange(1,hurs.shape[0]+1)
    
    ### Consolidates time, location, and variables of interest (i.e. temp) into one dataframe ###
    names = ['t','y','x']
    index = pd.MultiIndex.from_product([time,lats,lons], names=names)
    df = pd.DataFrame({'hurs': hurs.flatten()}, index=index)['hurs']
    df.index.names = ['time', 'lats', 'lons']
    df = df.reset_index(level=['time', 'lats', 'lons'])
    df = pd.DataFrame.to_numpy(df)
   

    # manage data for the state of indiana here
    
   
    
    ### coordinates from locations are in the following order: W, E, S, N
    W, E, S, N = state_locations.get_state_loc('IN')
    
    ### create matrix for the coordinates from the state lat = vertical (north to south), lon = horizontal (west to east)
    
        
    
    # Adds temperature columns (one for each timestamp) # 
    nlons = lons.shape[0]
    nlats = lats.shape[0]
    blockt0 = nlons*nlats
    
    
    
    # Create temp matrix with the columns for each time + appends to df_new #
    df_new = df[0:blockt0,1:3]
    for count, value in enumerate(time,start=0):    
        t_append = df[(count)*blockt0:(count+1)*blockt0,3]
        df_new = np.column_stack((df_new,t_append))
    
    
    
    return df_new, time, lats, lons, nlons '''

'#### Loads MERRA netcdf file and important variables ####\n# Note: operates over one day of data based on the daily nature of each MERRA file # \ndef organize_MERRAvar(MERRA_file_path):\n    data = Dataset(MERRA_file_path)\n    \n    ### Extracts location variables and other important variables from "data" ###\n    lons = data.variables[\'lon\'][:]\n    lats = data.variables[\'lat\'][:]\n    hurs = data.variables[\'hurs\'][:,:,:]\n    \n    # Arturo se la come\n    \n    ### print the variables saved on the dataset\n    print(data.variables)\n    \n    ### Defines time variable ###\n    \n    time = np.arange(1,hurs.shape[0]+1)\n    \n    ### Consolidates time, location, and variables of interest (i.e. temp) into one dataframe ###\n    names = [\'t\',\'y\',\'x\']\n    index = pd.MultiIndex.from_product([time,lats,lons], names=names)\n    df = pd.DataFrame({\'hurs\': hurs.flatten()}, index=index)[\'hurs\']\n    df.index.names = [\'time\', \'lats\', \'lons\']\n    df = df.reset_index(le

In [22]:

# Just the first path to test 
paths = [ 'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4']


# All the paths
'''paths = [ 'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4'
         ,'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20110101-20201231.nc4'
         ,'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20210101-20301231.nc4'
         ,'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20310101-20401231.nc4'
         ,'ISIMIP2b/SecondaryInputData/GCM_atmosphere/biascorrected/global/rcp45/HadGEM2-ES/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20410101-20501231.nc4']'''


# Get each nc4 file
i = 1  # Counter
for path in paths:
    # Open the netCDF4 file and read the data
    globals()[f"dataset_{i}"] = get__isimip_data(path)
    
    i =+ 1
    
print(dataset_1)
    
    
    
    



KeyboardInterrupt: 

In [ ]:
'''organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")'''

'organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")'